In [1]:
import pandas as pd
import numpy as np
import ast
 

In [2]:
df=pd.read_csv('rental_offers2.csv',index_col='index')
df.head()

,Unnamed: 0,Name,Features,Price,Agent
index,,,,,
0,0,"Piso de dos habitaciones Almirante Cadarso, Gr...","['2 habitaciones', '79 m²', '1 baño', 'Planta ...","€ 1.600/mes€ 1.700(-5,9%)",Alain Consultores Inmobiliarios
1,1,"Piso de cuatro habitaciones Maximilià Thous, S...","['4 habitaciones', '125 m²', '2 baños', 'Plant...",€ 1.300/mes,R&B Conforthouse
2,2,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '80 m²', '1 baño', 'Planta ...",€ 1.100/mes,R&B Conforthouse
3,3,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '96 m²', '2 baños', 'Planta...",€ 1.195/mes,R&B Conforthouse
4,4,"Piso tercera planta, Sant Francesc, València","['5 habitaciones', '236 m²', '3 baños', 'Plant...",€ 4.200/mes,PRIMERA AVENIDA - GRUPO INMOBILIARIO


In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)

df.drop_duplicates(inplace=True)

df['House_type']=df['Name'].apply(lambda x: x.split(' ')[0])
df['House_type'].replace({'Dúplex,':'Dúplex'},inplace=True)

df['Location']=df['Name'].apply(lambda x: x.split(',')[-2].strip())

df['Features2']=df['Features'].apply(lambda x: ast.literal_eval(x))

def furnished(x):
    if 'Amueblado' in x:
        return 1
    elif 'Parcialmente amueblado' in x:
        return 1
    else:
        return 0
df['Furnished']=df['Features2'].apply(lambda x: furnished(x))

def elevator(x):
    if 'Ascensor' in x:
        return 1
    elif 'No Ascensor' in x:
        return 0
    else:
        return np.nan
df['Elevator']=df['Features2'].apply(lambda x: elevator(x))

def terrace(x):
    if 'Terraza' in x:
        return 1
    else:
        return 0
def balcony(x):
    if 'Balcón' in x:
        return 1
    else:
        return 0
df['Terrace']=df['Features2'].apply(lambda x: terrace(x))
df['Balcony']=df['Features2'].apply(lambda x: balcony(x))

def storage(x):
    if 'Trastero' in x:
        return 1
    else:
        return 0
df['Storage']=df['Features2'].apply(lambda x: storage(x))

def rooms(x):
    for i in x:
        if 'habitaci' in i:
            return i.split()[0]
    return np.nan
df['Rooms']=df['Features2'].apply(lambda x: rooms(x))

def bathrooms(x):
    for i in x:
        if 'baño' in i:
            return i.split()[0]
    return np.nan
df['Bathrooms']=df['Features2'].apply(lambda x: bathrooms(x))

def floor(x):
    for i in x:
        if 'Planta' in i:
            return i.split()[1]
    return np.nan
df['Floor']=df['Features2'].apply(lambda x: floor(x))

def surface(x):
    for i in x:
        if 'm²' in i:
            return int(i.split()[0].replace('.',''))
    return np.nan
df['Surface']=df['Features2'].apply(lambda x: surface(x))

df['Price2']=df['Price'].apply(lambda x: x.split(' ')[1].split('/')[0].replace('.',''))

#print(df.loc[df.Price2=='a'])
df.drop(659,inplace=True)
df.drop(723,inplace=True)
#print(df.loc[df.Price2=='a'])

df['Price2']=df['Price'].apply(lambda x: int(x.split(' ')[1].split('/')[0].replace('.','')))

df.drop(['Name','Features','Price','Features2'],axis=1,inplace=True)
df.rename({'Price2':'Price'},axis=1,inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703 entries, 0 to 722
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Agent       703 non-null    object 
 1   House_type  703 non-null    object 
 2   Location    703 non-null    object 
 3   Furnished   703 non-null    int64  
 4   Elevator    656 non-null    float64
 5   Terrace     703 non-null    int64  
 6   Balcony     703 non-null    int64  
 7   Storage     703 non-null    int64  
 8   Rooms       642 non-null    object 
 9   Bathrooms   669 non-null    object 
 10  Floor       309 non-null    object 
 11  Surface     514 non-null    float64
 12  Price       703 non-null    int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 76.9+ KB


In [4]:
df.drop(list(df.loc[df.House_type=='Edificio'].index),inplace=True)

df['Floor'].replace({'B':'0'},inplace=True)
df['Floor']=pd.to_numeric(df['Floor'],errors='coerce').astype('Int32')
df.loc[df['Floor']==0,'Elevator']=df.loc[df['Floor']==0,'Elevator'].fillna(0)
df['Elevator'].fillna(df['Elevator'].mode()[0],inplace=True)

df.drop(df.loc[(df.Surface.isna()) & (df.Rooms.isna())].index,inplace=True)

df['Rooms'].replace({'5+':'6'},inplace=True)
df['Rooms']=pd.to_numeric(df['Rooms'],errors='coerce').astype('Int32')

ratio_surf_room=df['Surface'].mean()/df['Rooms'].mean()
df['Rooms'].fillna(round(df['Surface']/ratio_surf_room),inplace=True)
df['Rooms'].replace({0:1},inplace=True)

df.drop(df.loc[(df.Surface.isna()) & (df.Bathrooms.isna())].index,inplace=True)

df['Bathrooms'].replace({'3+':'4'},inplace=True)
df['Bathrooms']=pd.to_numeric(df['Bathrooms'],errors='coerce').astype('Int32')

ratio_surf_bathroom=df['Surface'].mean()/df['Bathrooms'].mean()
df['Bathrooms'].fillna(round(df['Surface']/ratio_surf_bathroom),inplace=True)
df['Bathrooms'].replace({0:1},inplace=True)

df.drop('Floor',axis=1,inplace=True)

ratio_surf_all=df['Surface'].mean()/(df['Rooms'].mean()
                                     +0.5*df['Bathrooms'].mean()
                                     +0.75*df['Terrace'].mean()
                                     +0.25*df['Balcony'].mean())
df['Surface'].fillna(round(ratio_surf_all*(df['Rooms']+0.5*df['Bathrooms']+0.75*df['Terrace']+0.25*df['Balcony'])),inplace=True)

df['Elevator']=df['Elevator'].astype('int')
df['Surface']=df['Surface'].astype('int')

df.drop(df.loc[df['Price']<600].index,inplace=True)

group_loc=pd.DataFrame(df['Location'].value_counts()).reset_index()
grouped_locations=list(group_loc.loc[group_loc['count']<5]['Location'].unique())
df['Location'].replace(grouped_locations,['Other' for i in range(len(grouped_locations))],inplace=True)

df['House_type'].replace(['Casa','Dúplex','Chalet'],['Other','Other','Other'],inplace=True)

In [5]:
df.head()

,Agent,House_type,Location,Furnished,Elevator,Terrace,Balcony,Storage,Rooms,Bathrooms,Surface,Price
index,,,,,,,,,,,,
0,Alain Consultores Inmobiliarios,Piso,Gran Via,1,1,0,0,0,2,1,79,1600
1,R&B Conforthouse,Piso,Sant Antoni,0,1,0,0,0,4,2,125,1300
2,R&B Conforthouse,Piso,El Perellonet,0,1,1,0,0,3,1,80,1100
3,R&B Conforthouse,Piso,El Perellonet,0,1,0,1,0,3,2,96,1195
4,PRIMERA AVENIDA - GRUPO INMOBILIARIO,Piso,Sant Francesc,0,1,0,0,0,5,3,236,4200


In [6]:
df.to_csv('data2.csv')